# Convert list of values and dates into dekads

In [1]:
import pandas as pd
import numpy as np

begin = pd.datetime(2002,7,1)
end = pd.datetime(2016,12,31)

dtrange = pd.date_range(begin, end)

p1 = np.random.rand(len(dtrange)) + 5
p2 = np.random.rand(len(dtrange)) + 10

df = pd.DataFrame({'p1': p1, 'p2': p2}, index=dtrange)

In [2]:
type(p1)

numpy.ndarray

In [3]:
d = df.index.day - np.clip((df.index.day-1) // 10, 0, 2)*10 - 1
date = df.index.values - np.array(d, dtype="timedelta64[D]")
df.groupby(date).mean()

,p1,p2
2002-07-01,5.503441,10.529736
2002-07-11,5.481668,10.364516
2002-07-21,5.425301,10.429303
2002-08-01,5.487707,10.432215
2002-08-11,5.584481,10.447301
...,...,...
2016-11-11,5.464668,10.317146
2016-11-21,5.567298,10.382041
2016-12-01,5.458229,10.554661
2016-12-11,5.454361,10.409165


In [7]:
date

array(['2002-07-01T00:00:00.000000000', '2002-07-01T00:00:00.000000000',
       '2002-07-01T00:00:00.000000000', ...,
       '2016-12-21T00:00:00.000000000', '2016-12-21T00:00:00.000000000',
       '2016-12-21T00:00:00.000000000'], dtype='datetime64[ns]')

In [5]:
df.index.dayofyear

Int64Index([182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
            ...
            357, 358, 359, 360, 361, 362, 363, 364, 365, 366],
           dtype='int64', length=5298)

# Resample daily files in folders to dekads

In [58]:
import rasterio
import numpy as np
from glob import glob
import os
from rasterio.plot import show

In [63]:
def read_file(file):
    with rasterio.open(file) as src:
        return(src.read())
    
def dekad_resample(root_dir, out_dir, var):
    folds = [fold for fold in sorted(os.listdir(root_dir))]
    for fold in folds:
        os.chdir(root_dir + fold + '/')
        files = [file for file in sorted(os.listdir(os.getcwd()))]
        dekad1 = [read_file(x) for x in files[0:10]]
        dekad2 = [read_file(x) for x in files[10:20]]
        dekad3 = [read_file(x) for x in files[20:]]
        
     # Perform averaging
        dekad1_out = np.mean(dekad1, axis=0)
        dekad2_out = np.mean(dekad2, axis=0)
        dekad3_out = np.mean(dekad3, axis=0)

        with rasterio.open(files[0]) as src:
            meta = src.meta

        meta.update(dtype=rasterio.float32)

        # Write output file
        with rasterio.open(out_dir + var + fold + '_01.tif', 'w', **meta) as dst:
            dst.write(dekad1_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_02.tif', 'w', **meta) as dst:
            dst.write(dekad2_out.astype(rasterio.float32))

        with rasterio.open(out_dir + var + fold + '_03.tif', 'w', **meta) as dst:
            dst.write(dekad3_out.astype(rasterio.float32))


In [64]:
tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Months/' 
out_tmax_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRTS/Dekads/'
sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Months/' 
out_sm_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Dekads/' 

In [65]:
dekad_resample(sm_dir, out_sm_dir, var = 'SM_')